In [155]:
import cadquery as cq
from cadquery import exporters
import ocp_vscode
import ocp_vscode.backend

wall = 1.2
stud_radius = 9.5 / 2
stud_spacing = 16
stud_height = 3.75
stud_thickness = wall
base_shrink = 0.2
inner_cylinder_radius = 6.65
inner_cylinder_thickness = wall - 0.2
unit_thickness = 9.5
ledge_thickness = 1.0
ledge_length = 3.2

def make_rectangule(height, width, thickness) -> cq.Workplane:
    base_height = stud_spacing * height - base_shrink * 2
    base_width = stud_spacing * width - base_shrink * 2
    base_height_inner = base_height - wall * 2
    base_width_inner = base_width - wall * 2
    base_thickness = unit_thickness * thickness - 0.1
    sketch: cq.Sketch = cq.Workplane().sketch().rect(base_height, base_width)
    sketch = sketch.rect(base_height_inner, base_width_inner, mode="s")
    
    # side
    x = stud_spacing * (height / 2 - 1)
    y = stud_spacing * (width / 2 - 1)
    sketch = sketch.rarray(stud_spacing, stud_spacing, height-1, width-1)
    sketch = sketch.circle(inner_cylinder_radius)
    sketch = sketch.circle(inner_cylinder_radius - inner_cylinder_thickness, "s")
    result: cq.Workplane = sketch.finalize()
    result = result.extrude(base_thickness).tag("base")
    
    result = result.faces(">Z").workplane(offset=-wall)
    result = result.box(base_height, base_width, wall, [True, True, False])
    
    sketch = result.faces(">Z").workplane().sketch()
    sketch = sketch.rarray(stud_spacing, stud_spacing, height, width)
    result = sketch.circle(stud_radius).circle(stud_radius - stud_thickness, "s").finalize()
    result = result.extrude(stud_height).tag("stud")

    
    ledgeX: cq.Workplane = cq.Workplane().sketch()
    ledgeX = ledgeX.rarray(base_height - ledge_length, stud_spacing, 2, width)
    ledgeX = ledgeX.rect(ledge_length, ledge_thickness).finalize()
    ledgeX = ledgeX.extrude(base_thickness, False).tag("ledgeX").edges("|Y").chamfer(2, 0.2)
    
    ledgeY: cq.Workplane = cq.Workplane().sketch()
    ledgeY = ledgeY.rarray(stud_spacing, base_width - ledge_length, height, 2)
    ledgeY = ledgeY.rect(ledge_thickness, ledge_length).finalize()
    ledgeY = ledgeY.extrude(base_thickness, False).tag("ledgeY").edges("|X").chamfer(2, 0.2)
    
    result = result.add(ledgeX).add(ledgeY).combine()
    
    # Add fillets to the base edges
    result = result.faces(">X or <X or >Y or <Y").edges().fillet(0.6)
    result = result.faces(">Z").edges().fillet(0.4)
    return result

def main():
    result = make_rectangule(2, 4, 1)
    exporters.export(result, "plate2x4.stl")
    ocp_vscode.show_object(result, clear=True)
    
main()

+